In [1]:
import requests
import pandas as pd

In [ ]:
INDICATOR_CODES = ['SP.POP.TOTL', # Total Population
                   'SP.POP.TOTL.FE.IN', # Female Population
                   'SP.POP.TOTL.MA.IN', # Male Population
                   'SP.DYN.CBRT.IN', # Birth Rate
                   'SP.DYN.CDRT.IN', # Death Rate
                   'SE.COM.DURS', # Compulsory Education Dur.
                   'SL.IND.EMPL.ZS', # Employment in Industry(%)
                   'SL.AGR.EMPL.ZS', # Employment in Agriculture(%)
                   'SL.AGR.EMPL.FE.ZS', # Female Employment in Agriculture(%)
                   'SL.IND.EMPL.FE.ZS', # Female Employment in Industry(%)
                   'SL.UEM.TOTL.ZS', # Unemployment(%)
                   'NY.GDP.MKTP.CD', # GDP in USD
                   'NY.ADJ.NNTY.PC.KD.ZG', # National Income per Capita
                   'NY.GSR.NFCY.CD', # Net income from Abroad
                   'NV.AGR.TOTL.CD', # Agriculture value added(in USD)
                   'EG.USE.ELEC.KH.PC', # Electric Power Consumption(kWH per capita)
                   'EG.FEC.RNEW.ZS', # Renewable Energy Consumption (%)
                   'EG.USE.COMM.FO.ZS', # Fossil Fuel Consumption (%)
                   'SP.DYN.LE00.MA.IN', # Male life expectancy
                   'SP.DYN.LE00.FE.IN', # Female life expectancy
                   'SE.PRM.ENRR', # School enrollment, primary
                   'SE.TER.ENRR', # School enrollment, tertiary
                   'SE.PRM.CMPT.ZS', # Primary completion rate
                   'SE.ADT.1524.LT.ZS' # Literacy rate
                   ]

In [3]:
COUNTRY_CODES = ['AUS', 'BGD', 'CAN', 'CHN', 'DEU', 'GBR', 'IDN', 'IND', 'JPN', 'MMR', 'MEX', 'PHL', 'RUS', 'THA', 'USA', 'VNM']

In [4]:
def loadData(country_code, start, end, format = "json", per_page = 100):

    records = {}
    for indicator_code in INDICATOR_CODES:
        url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/{indicator_code}?date={start}:{end}&format=json&per_page={per_page}"
        response = requests.get(url)

        if response.status_code == 200:

            data = response.json()

            count = data[0]['total']
            
            for i in range(count):
                item = data[1][i]
                feature = item['indicator']['value']
                country = item['country']['value']
                year = item['date']
                value = item['value']

                if year not in records:
                    records[year] = {"Year": year}
                
                records[year]['Country'] = country

                records[year][feature] = value
            

        df = pd.DataFrame(records.values())

    return df
        
    

In [5]:
raw_data = pd.DataFrame()
for country in COUNTRY_CODES:
    print(country)

    df = loadData(country, 2000, 2020)
    raw_data = pd.concat([raw_data, df], ignore_index=True)

AUS
BGD


C:\Users\ndh01\AppData\Local\Temp\ipykernel_6596\2178919890.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  raw_data = pd.concat([raw_data, df], ignore_index=True)


CAN
CHN
DEU
GBR
IDN
IND
JPN
MMR
MEX
PHL
RUS
THA
USA
VNM


In [6]:
raw_data.head()

,Year,Country,"Population, total","Population, female","Population, male","Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)","Compulsory education, duration (years)",Employment in industry (% of total employment) (modeled ILO estimate),Employment in agriculture (% of total employment) (modeled ILO estimate),...,"Agriculture, forestry, and fishing, value added (current US$)",Electric power consumption (kWh per capita),Renewable energy consumption (% of total final energy consumption),Fossil fuel energy consumption (% of total),"Life expectancy at birth, male (years)","Life expectancy at birth, female (years)","School enrollment, primary (% gross)","School enrollment, tertiary (% gross)","Primary completion rate, total (% of relevant age group)","Literacy rate, youth total (% of people ages 15-24)"
0,2020,Australia,25649248,12922405,12726843,11.5,6.3,11.0,19.191097,2.765998,...,2.673100e+10,NaN,11.2,NaN,81.2,85.3,99.901108,110.139221,NaN,NaN
1,2019,Australia,25334826,12762303,12572523,12.1,6.7,11.0,19.058460,2.558293,...,2.946698e+10,NaN,10.3,NaN,80.9,85.0,100.199699,109.759560,NaN,NaN
2,2018,Australia,24963258,12575335,12387923,12.6,6.3,11.0,19.887638,2.616852,...,3.512464e+10,NaN,9.8,NaN,80.7,84.9,100.398712,103.322090,NaN,NaN
3,2017,Australia,24592588,12390298,12202290,12.6,6.5,11.0,19.430111,2.596054,...,3.587619e+10,NaN,9.9,NaN,80.5,84.6,100.637657,109.993309,NaN,NaN
4,2016,Australia,24190907,12185876,12005031,12.9,6.6,11.0,19.695735,2.621225,...,2.903110e+10,NaN,9.6,NaN,80.4,84.6,101.180687,119.682777,NaN,NaN
